# 네이버 증권 자료 다운받기

# dart 자료와 네이버 증권 자료 병합하기

In [ ]:
# dart 자료에서 특정 index삭제하여 2021 분기 자료 삭제
f=case.finance
f.reset_index(drop=True,inplace=True)
indexName = f[f['period']==202109].index
f.drop(indexName,inplace=True)

In [ ]:
# 분기 자료에 단위 맞춰서 더하고 저장하기
with open('Raw_Finance/finance_naver_converted_202103.pickle', 'rb') as handle:
    naver1 = pickle.load(handle)
for col in naver1.columns[2:]:
    naver1[col] = naver1[col]*100000000

naver2 = case.finance_naver
for col in naver2.columns[2:]:
    naver2[col] = naver2[col]*100000000
    
finance = pd.concat([case.finance,naver1])
finance = pd.concat([finance,naver2])
finance.reset_index(drop=True,inplace=True)

with open('Raw_Finance/finance_latest.pickle', 'wb') as handle:
    pickle.dump(finance, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 3 차원 그래프, 히트맵으로 성과 확인하기

In [ ]:
AxisSell = []
AxisBuy = []
AxisCAGR = []
AxisMDD = []
AxisMAR = []
AxisSharpe = []
AxisnoOfTrade = []
case.daysContinuous = 5
for buy in range(10,61,5):
    for sell in range(10,61,5):
        test1 = case.applyTraidingStrategy(strategy="MAKosdaqContinuous",daysForMABuy=buy,daysForMASell=sell)
        AxisSell.append(sell)
        AxisBuy.append(buy)
        AxisnoOfTrade.append(case.noOfTrade)
        result = case.simple_analyzer(test1)
        AxisCAGR.append(result.at['CAGR','BuyMA'])
        AxisMDD.append(result.at['MDD','BuyMA']*(-1))
        AxisSharpe.append(result.at['Sharpe_Ratio','BuyMA'])
        AxisMAR.append(result.at['CAGR','BuyMA'] / result.at['MDD','BuyMA']*(-1))

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(AxisBuy, AxisSell, AxisMAR, marker='o', s=15, cmap='Greens')
plt.xlabel('Buy')
plt.ylabel('Sell')
plt.show()

In [ ]:
loss = 0.5  # 잦은 거래로 인한 손실
AxisCAGRadjusted = [a*((1-loss/100)**(b/8.8+4)) for a,b in zip(AxisCAGR, AxisnoOfTrade)]
df = pd.DataFrame({'buy':AxisBuy,'sell':AxisSell,'CAGR':AxisCAGR,'CAGRadj':AxisCAGRadjusted,'MDD':AxisMDD,'sharpe_ratio':AxisSharpe,
                   'MAR':AxisMAR,'noOfTrade':AxisnoOfTrade})

In [ ]:
df = pd.DataFrame({'buy':AxisBuy,'sell':AxisSell,'CAGR':AxisCAGR,'MDD':AxisMDD,'sharpe_ratio':AxisSharpe,
                   'MAR':AxisMAR,'noOfTrade':AxisnoOfTrade})
import seaborn as sns
import matplotlib.pyplot as plt
for i in df.columns[2:]:
    plt.figure(figsize=(7,7))
    plt.title(i)
    if i != 'noOfTrade':
        sns.heatmap(data = df.pivot_table(index=df.columns[0],columns=df.columns[1],values=i), annot=True,fmt = '.2f', linewidths=.2, cmap='Blues')
    else:
        sns.heatmap(data = df.pivot_table(index=df.columns[0],columns=df.columns[1],values=i), annot=True,fmt = '.0f', linewidths=.2, cmap='Blues')
    plt.savefig(f'image/{i}.png')

# 거래빈도 검토

In [ ]:
listCount = []
count = 1
listSignal = list(test['MASignal'])
for i in range(len(listSignal)-1):
    if listSignal[i] == listSignal[i+1]:
        count += 1
    else:
        listCount.append(count)
        count = 1
        
import matplotlib.pyplot as plt
D = dict(Counter(listCount).most_common())
plt.bar(list(D.keys()), list(D.values()),align='edge')

plt.plot(listCount)